In [212]:
import keras
import warnings
import numpy as np
import pandas as pd
from math import sqrt
from sklearn import metrics
from keras.optimizers import Adam
from sklearn.cluster import KMeans
from keras.models import load_model
from scipy.sparse.linalg import svds
from models import matrix_factorization_utilities
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation, Flatten ,Embedding, Reshape, concatenate
warnings.filterwarnings('ignore')

###  1. Data Load

In [152]:
# Reading the ratings data
ratings = pd.read_csv('Dataset/ratings.csv')

In [153]:
ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
1048570,7120,168,5.0,1175543061
1048571,7120,253,4.0,1175542225
1048572,7120,260,5.0,1175542035
1048573,7120,261,4.0,1175543376


In [154]:
ratings = ratings[['userId', 'movieId','rating']]

In [155]:
ratings

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
...,...,...,...
1048570,7120,168,5.0
1048571,7120,253,4.0
1048572,7120,260,5.0
1048573,7120,261,4.0


In [156]:
    #reading the movies dataset
    movies = pd.read_csv('Dataset/movies.csv')


In [157]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [158]:
#reading the links dataset
links = pd.read_csv('Dataset/movies.dat',sep='\t', encoding='latin-1')

In [159]:
links

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10192,65088,Bedtime Stories,960731,Más allá de los sueños,http://ia.media-imdb.com/images/M/MV5BMjA5Njk5...,2008,bedtime_stories,4.4,104,26,...,25,4.7,26,6,20,23,3.5,108877,63,http://content6.flixster.com/movie/10/94/33/10...
10193,65091,Manhattan Melodrama,25464,El enemigo público número 1,http://ia.media-imdb.com/images/M/MV5BMTUyODE3...,1934,manhattan_melodrama,7,12,10,...,83,0,4,2,2,50,3.7,344,71,http://content9.flixster.com/movie/66/44/64/66...
10194,65126,Choke,1024715,Choke,http://ia.media-imdb.com/images/M/MV5BMTMxMDI4...,2008,choke,5.6,135,73,...,54,4.9,26,8,18,30,3.3,13893,55,http://content6.flixster.com/movie/10/85/09/10...
10195,65130,Revolutionary Road,959337,Revolutionary Road,http://ia.media-imdb.com/images/M/MV5BMTI2MzY2...,2008,revolutionary_road,6.7,194,133,...,68,6.9,36,25,11,69,3.5,46044,70,http://content8.flixster.com/movie/10/88/40/10...


In [160]:
#drop link dataframe
for i in links.columns:
    if i=='id'or i == 'title'or i=='year' or i == 'rtPictureURL':
        continue
    else:
        links.drop(i, inplace=True, axis=1)

In [161]:
links

,id,title,year,rtPictureURL
0,1,Toy story,1995,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,1995,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,1993,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,1995,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,1995,http://content8.flixster.com/movie/25/54/25542...
...,...,...,...,...
10192,65088,Bedtime Stories,2008,http://content6.flixster.com/movie/10/94/33/10...
10193,65091,Manhattan Melodrama,1934,http://content9.flixster.com/movie/66/44/64/66...
10194,65126,Choke,2008,http://content6.flixster.com/movie/10/85/09/10...
10195,65130,Revolutionary Road,2008,http://content8.flixster.com/movie/10/88/40/10...


In [162]:
links.columns=['movieId','title_old', 'year', 'rtPictureURL'] # columns rename

In [163]:
links

,movieId,title_old,year,rtPictureURL
0,1,Toy story,1995,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,1995,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,1993,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,1995,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,1995,http://content8.flixster.com/movie/25/54/25542...
...,...,...,...,...
10192,65088,Bedtime Stories,2008,http://content6.flixster.com/movie/10/94/33/10...
10193,65091,Manhattan Melodrama,1934,http://content9.flixster.com/movie/66/44/64/66...
10194,65126,Choke,2008,http://content6.flixster.com/movie/10/85/09/10...
10195,65130,Revolutionary Road,2008,http://content8.flixster.com/movie/10/88/40/10...


In [164]:
# title_old and year formating merge and add new columns
links['title']=links.apply(lambda x:'%s (%s)' % (x['title_old'],x['year']),axis=1)

In [165]:
links

,movieId,title_old,year,rtPictureURL,title
0,1,Toy story,1995,http://content7.flixster.com/movie/10/93/63/10...,Toy story (1995)
1,2,Jumanji,1995,http://content8.flixster.com/movie/56/79/73/56...,Jumanji (1995)
2,3,Grumpy Old Men,1993,http://content6.flixster.com/movie/25/60/25602...,Grumpy Old Men (1993)
3,4,Waiting to Exhale,1995,http://content9.flixster.com/movie/10/94/17/10...,Waiting to Exhale (1995)
4,5,Father of the Bride Part II,1995,http://content8.flixster.com/movie/25/54/25542...,Father of the Bride Part II (1995)
...,...,...,...,...,...
10192,65088,Bedtime Stories,2008,http://content6.flixster.com/movie/10/94/33/10...,Bedtime Stories (2008)
10193,65091,Manhattan Melodrama,1934,http://content9.flixster.com/movie/66/44/64/66...,Manhattan Melodrama (1934)
10194,65126,Choke,2008,http://content6.flixster.com/movie/10/85/09/10...,Choke (2008)
10195,65130,Revolutionary Road,2008,http://content8.flixster.com/movie/10/88/40/10...,Revolutionary Road (2008)


In [166]:
# drop old columns
links.drop('title_old', inplace=True, axis=1)
links.drop('year', inplace=True, axis=1)

In [167]:
links

,movieId,rtPictureURL,title
0,1,http://content7.flixster.com/movie/10/93/63/10...,Toy story (1995)
1,2,http://content8.flixster.com/movie/56/79/73/56...,Jumanji (1995)
2,3,http://content6.flixster.com/movie/25/60/25602...,Grumpy Old Men (1993)
3,4,http://content9.flixster.com/movie/10/94/17/10...,Waiting to Exhale (1995)
4,5,http://content8.flixster.com/movie/25/54/25542...,Father of the Bride Part II (1995)
...,...,...,...
10192,65088,http://content6.flixster.com/movie/10/94/33/10...,Bedtime Stories (2008)
10193,65091,http://content9.flixster.com/movie/66/44/64/66...,Manhattan Melodrama (1934)
10194,65126,http://content6.flixster.com/movie/10/85/09/10...,Choke (2008)
10195,65130,http://content8.flixster.com/movie/10/88/40/10...,Revolutionary Road (2008)


In [168]:
#replace columns
links=links[['movieId','title','rtPictureURL']]

In [169]:
links

,movieId,title,rtPictureURL
0,1,Toy story (1995),http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji (1995),http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men (1993),http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale (1995),http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II (1995),http://content8.flixster.com/movie/25/54/25542...
...,...,...,...
10192,65088,Bedtime Stories (2008),http://content6.flixster.com/movie/10/94/33/10...
10193,65091,Manhattan Melodrama (1934),http://content9.flixster.com/movie/66/44/64/66...
10194,65126,Choke (2008),http://content6.flixster.com/movie/10/85/09/10...
10195,65130,Revolutionary Road (2008),http://content8.flixster.com/movie/10/88/40/10...


In [170]:
# merge dataframe 
movie_list=pd.merge(movies,links, on='movieId')

In [171]:
# drop column
movie_list.drop('title_y', inplace=True, axis=1)

In [172]:
# rename columns
movie_list.columns=['movieId','title','genres','rtPictureURL']

In [173]:
ratings.userId

0             1
1             1
2             1
3             1
4             1
           ... 
1048570    7120
1048571    7120
1048572    7120
1048573    7120
1048574    7120
Name: userId, Length: 1048575, dtype: int64

In [174]:
# convert type category
ratings.userId = ratings.userId.astype('category').cat.codes.values

In [175]:
ratings.userId

0             0
1             0
2             0
3             0
4             0
           ... 
1048570    7119
1048571    7119
1048572    7119
1048573    7119
1048574    7119
Name: userId, Length: 1048575, dtype: int16

In [176]:
ratings.userId.max()

7119

In [177]:
ratings.movieId = ratings.movieId.astype('category').cat.codes.values

In [178]:
ratings.movieId

0            1
1           28
2           31
3           46
4           49
          ... 
1048570    163
1048571    247
1048572    254
1048573    255
1048574    260
Name: movieId, Length: 1048575, dtype: int16

In [179]:
ratings.movieId.max()

14025

In [180]:
#take 80% as the training set and 20% as the test set
train, test= train_test_split(ratings,test_size=0.2)

In [181]:
train

,userId,movieId,rating
117107,782,628,3.5
48900,366,1280,3.5
574484,3857,13495,2.5
588535,3943,350,4.5
286098,1965,5142,1.0
...,...,...,...
971131,6544,1120,4.5
855154,5724,6277,3.0
217894,1506,8392,0.5
899827,6033,1888,2.0


In [182]:
test

,userId,movieId,rating
200665,1373,1142,5.0
935717,6270,2006,2.0
576467,3867,2354,5.0
73906,520,7773,3.5
367556,2486,2124,3.5
...,...,...,...
876813,5846,294,5.0
233432,1607,1911,3.0
469764,3196,4641,4.0
583545,3906,12234,3.0


### 2.Matrix Factorisation Model

#### 2.1. Matrix Factorisation Model Calculate Best n_latent

**Latent features:** At the expense of over-simplication, latent features are 'hidden' features to distinguish them from observed features. Latent features are computed from observed features using matrix factorization. An example would be text document analysis. 'words' extracted from the documents are features.

In [183]:
n_users = ratings.userId.unique().shape[0]
n_items = ratings.movieId.unique().shape[0]
n_users, n_items

(7120, 14026)

In [184]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
    #for every line in the data
for line in train.itertuples():
    #set the value in the column and row to 
    #line[1] is userId, line[2] is movieId and line[3] is rating, line[4] is movie_index and line[5] is user_index
    train_data_matrix[line[1]-1,line[2]-1] = line[3]
train_data_matrix.shape

(7120, 14026)

In [185]:
#Create two user-item matrices, one for training and another for testing
test_data_matrix = np.zeros((n_users, n_items))
    #for every line in the data
for line in test.itertuples():
    #set the value in the column and row to 
    #line[1] is userId, line[2] is movieId and line[3] is rating, line[4] is movie_index and line[5] is user_index
    #print(line[2])
    test_data_matrix[line[1]-1,line[2]-1] = line[3]
test_data_matrix.shape

(7120, 14026)

In [186]:
pd.DataFrame(train_data_matrix).head()

,0,1,2,3,4,5,6,7,8,9,...,14016,14017,14018,14019,14020,14021,14022,14023,14024,14025
0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [187]:
train['rating'].max()

5.0

In [188]:
#Calculate the rmse sscore of SVD using different values of k (latent features)
rmse_list = []
mae_list = []
accuracy=[]
pred_list=[]
a=[1,2,5,10,20]
for i in a:
    #apply svd to the test data
    u,s,vt = svds(train_data_matrix,k=i)
    #get diagonal matrix
    s_diag_matrix=np.diag(s)
    #predict x with dot product of u s_diag and vt
    X_pred = np.dot(np.dot(u,s_diag_matrix),vt)
    pred_list.append(X_pred)
    #calculate rmse score of matrix factorisation predictions
    mse =mean_squared_error(test_data_matrix, X_pred)
    
    rmse_score = sqrt(mse)
    rmse_list.append(round(rmse_score,4))
    mae_score = mean_absolute_error(test_data_matrix,X_pred)
    mae_list.append(round(mae_score,4))

In [189]:
min_value_rmse=min(rmse_list)
min_index_rmse=rmse_list.index(min_value_rmse)
min_value_mae=min(mae_list)
min_index_mae=mae_list.index(min_value_mae)

print("Matrix Factorisation with " + str(a[min_index_rmse]) +" latent features has a RMSE of " + str(min_value_rmse))
print("Matrix Factorisation with " + str(a[min_index_mae]) +" latent features has a MAE of " + str(min_value_mae))

Matrix Factorisation with 1 latent features has a RMSE of 0.1948
Matrix Factorisation with 1 latent features has a MAE of 0.0389


In [190]:
movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(14026 + 1, 1, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(7120 + 1, 1,name='User-Embedding')(user_input))
prod = keras.layers.concatenate([movie_vec, user_vec],name='DotProduct')
result = keras.layers.Dense(1, activation='relu',name='Activation')(prod)
adam = Adam(lr=0.005)
model = keras.Model([user_input, movie_input], result)
model.compile(optimizer=adam,loss= 'mean_absolute_error')

```
keras.Input(
    shape=None,
    batch_size=None,
    name=None,
    dtype=None,
    sparse=None,
    tensor=None,
    ragged=None,
    type_spec=None,
    **kwargs
)
```
##### Arguments

* shape: A shape tuple (integers), not including the batch size. For instance, shape=(32,) indicates that the expected input will be batches of 32-dimensional vectors. Elements of this tuple can be None; 'None' elements represent dimensions where the shape is not known.

* batch_size: optional static batch size (integer).

* name: An optional name string for the layer. Should be unique in a model (do not reuse the same name twice). It will be autogenerated if it isn't provided.
 
* dtype: The data type expected by the input, as a string (float32, float64, int32...)

* sparse: A boolean specifying whether the placeholder to be created is sparse. Only one of 'ragged' and 'sparse' can be True. Note that, if sparse is False, sparse tensors can still be passed into the input - they will be densified with a default value of 0.

* tensor: Optional existing tensor to wrap into the Input layer. If set, the layer will use the tf.TypeSpec of this tensor rather than creating a new placeholder tensor.

* ragged: A boolean specifying whether the placeholder to be created is ragged. Only one of 'ragged' and 'sparse' can be True. In this case, values of 'None' in the 'shape' argument represent ragged dimensions. For more information about RaggedTensors, see this guide.

* type_spec: A tf.TypeSpec object to create the input placeholder from. When provided, all other args except name must be None.

* **kwargs: deprecated arguments support. Supports batch_shape and batch_input_shape.



In [191]:
# girdi boyutunu belirler movies
movie_input = keras.layers.Input(shape=[1],name='Item')

```
tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs
)
```

#### Arguments

* input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
* output_dim: Integer. Dimension of the dense embedding.
* embeddings_initializer: Initializer for the embeddings matrix (see keras.initializers).
* embeddings_regularizer: Regularizer function applied to the embeddings matrix (see keras.regularizers).
* embeddings_constraint: Constraint function applied to the embeddings matrix (see keras.constraints).
* mask_zero: Boolean, whether or not the input value 0 is a special "padding" value that should be masked out. This is useful when using recurrent layers which may take variable length input. If this is True, then all subsequent layers in the model need to support masking or an exception will be raised. If mask_zero is set to True, as a consequence, index 0 cannot be used in the vocabulary (input_dim should equal size of vocabulary + 1).
* input_length: Length of input sequences, when it is constant. This argument is required if you are going to connect Flatten then Dense layers upstream (without it, the shape of the dense outputs cannot be computed).

In [192]:
#input_dim = movie index +1
# output_dim = Dimension of the dense embedding.
movie_embedding = keras.layers.Embedding(14026 + 1, 1, name='Movie-Embedding')(movie_input)

```
tf.keras.layers.Flatten(data_format=None, **kwargs)
```

Flattens the input. Does not affect the batch size.

* Note: If inputs are shaped (batch,) without a feature axis, then flattening adds an extra channel dimension and output shape is (batch, 1).

#### Arguments

* data_format: A string, one of channels_last (default) or channels_first. The ordering of the dimensions in the inputs. channels_last corresponds to inputs with shape (batch, ..., channels) while channels_first corresponds to inputs with shape (batch, channels, ...). It defaults to the image_data_format value found in your Keras config file at ~/.keras/keras.json. If you never set it, then it will be "channels_last".

In [193]:
# Reshaping layers
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

In [194]:
# girdi boyutunu belirler users
user_input = keras.layers.Input(shape=[1],name='User')


Girdi verilerinin tamsayı olarak kodlanmasını gerektirir, böylece her kelime benzersiz bir tamsayı ile temsil edilir. Bu veri hazırlama adımı, Keras ile birlikte sağlanan Tokenizer API kullanılarak gerçekleştirilebilir.

Gömme katmanı rastgele ağırlıklarla başlatılır ve eğitim veri setindeki tüm kelimeler için bir yerleştirme öğrenir.

In [195]:

user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(7120 + 1, 1,name='User-Embedding')(user_input))


In [196]:
# merge e embeedding layer input

In [197]:
prod = keras.layers.concatenate([movie_vec, user_vec],name='DotProduct')


```
tf.keras.layers.Dense(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
```
#### Arguments

 * units: Positive integer, dimensionality of the output space.
 * activation: Activation function to use. If you don't specify anything, no activation is applied (ie. "linear" activation: a(x) = x).
 * use_bias: Boolean, whether the layer uses a bias vector.
 * kernel_initializer: Initializer for the kernel weights matrix.
 * bias_initializer: Initializer for the bias vector.
 * kernel_regularizer: Regularizer function applied to the kernel weights matrix.
 * bias_regularizer: Regularizer function applied to the bias vector.
 * activity_regularizer: Regularizer function applied to the output of the layer (its "activation").
 * kernel_constraint: Constraint function applied to the kernel weights matrix.
 * bias_constraint: Constraint function applied to the bias vector.

 etkinleştirme argümanı olarak iletilen öğe bazlı etkinleştirme işlevidir, çekirdek, katman tarafından oluşturulan bir ağırlık matrisidir ve önyargı, oluşturulan bir önyargı vektörüdür katmana göre (yalnızca use_bias True ise geçerlidir).

In [198]:
result = keras.layers.Dense(1, activation='relu',name='Activation')(prod)
# Eğittiğimiz yapay sinir ağı modeline gerçek dünya verilerini,
# yani karmaşık verileri öğretebilmek için kullandığımız fonksiyonlardır.
# relu =[0,+∞)
# ReLU fonksiyonunun ana avantajı aynı anda tüm nöronları aktive etmemesidir.

* Derin öğrenme uygulamalarında öğrenme işleminin temelde bir optimizasyon problemi olduğu daha önce vurgulanmıştı.
* Doğrusal olmayan problemlerin çözümünde optimum değeri bulmak için optimizasyon yöntemleri kullanılmaktadır. 

In [199]:
# lr =learning rate

In [200]:
adam = Adam(lr=0.05)


In [201]:
# creating model
model = keras.Model([user_input, movie_input], result)

```
Model.compile(
    optimizer="rmsprop",
    loss=None,
    metrics=None,
    loss_weights=None,
    weighted_metrics=None,
    run_eagerly=None,
    steps_per_execution=None,
    **kwargs
)
```
#### Arguments

* optimizer: String (name of optimizer) or optimizer instance. See tf.keras.optimizers.
* loss: String (name of objective function), objective function or tf.keras.losses.Loss instance. See tf.keras.losses. An objective function is any callable with the signature loss = fn(y_true, y_pred), where y_true = ground truth values with shape = [batch_size, d0, .. dN], except sparse loss functions such as sparse categorical crossentropy where shape = [batch_size, d0, .. dN-1]. y_pred = predicted values with shape = [batch_size, d0, .. dN]. It returns a weighted loss float tensor. If a custom Loss instance is used and reduction is set to NONE, return value has the shape [batch_size, d0, .. dN-1] ie. per-sample or per-timestep loss values; otherwise, it is a scalar. If the model has multiple outputs, you can use a different loss on each output by passing a dictionary or a list of losses. The loss value that will be minimized by the model will then be the sum of all individual losses.
* metrics: List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. See tf.keras.metrics. Typically you will use metrics=['accuracy']. A function is any callable with the signature result = fn(y_true, y_pred). To specify different metrics for different outputs of a multi-output model, you could also pass a dictionary, such as metrics={'output_a': 'accuracy', 'output_b': ['accuracy', 'mse']}. You can also pass a list (len = len(outputs)) of lists of metrics such as metrics=[['accuracy'], ['accuracy', 'mse']] or metrics=['accuracy', ['accuracy', 'mse']]. When you pass the strings 'accuracy' or 'acc', we convert this to one of tf.keras.metrics.BinaryAccuracy, tf.keras.metrics.CategoricalAccuracy, tf.keras.metrics.SparseCategoricalAccuracy based on the loss function used and the model output shape. We do a similar conversion for the strings 'crossentropy' and 'ce' as well.
* loss_weights: Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs. The loss value that will be minimized by the model will then be the weighted sum of all individual losses, weighted by the loss_weights coefficients. If a list, it is expected to have a 1:1 mapping to the model's outputs. If a dict, it is expected to map output names (strings) to scalar coefficients.
* weighted_metrics: List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
* run_eagerly: Bool. Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function. run_eagerly=True is not supported when using tf.distribute.experimental.ParameterServerStrategy.
* steps_per_execution: Int. Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead. At most, one full epoch will be run each execution. If a number larger than the size of the epoch is passed, the execution will be truncated to the size of the epoch. Note that if steps_per_execution is set to N, Callback.on_batch_begin and Callback.on_batch_end methods will only be called every N batches (i.e. before/after each tf.function execution).
* **kwargs: Arguments supported for backwards compatibility only.

In [202]:
model.compile(optimizer=adam,loss= 'mean_absolute_error', metrics=['accuracy'])

In [203]:
# model details
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
User (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
Movie-Embedding (Embedding)     (None, 1, 1)         14027       Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 1)         7121        User[0][0]                       
___________________________________________________________________________________________

```
Model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose="auto",
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
```

#### Arguments

* x: Input data. It could be:
  * A Numpy array (or array-like), or a list of arrays (in case the model has multiple inputs).
  * A TensorFlow tensor, or a list of tensors (in case the model has multiple inputs).
  * A dict mapping input names to the corresponding array/tensors, if the model has named inputs.
  * A tf.data dataset. Should return a tuple of either (inputs, targets) or (inputs, targets, sample_weights).
  * A generator or keras.utils.Sequence returning (inputs, targets) or (inputs, targets, sample_weights).
  * A tf.keras.utils.experimental.DatasetCreator, which wraps a callable that takes a single argument of type tf.distribute.InputContext, and returns a tf.data.Dataset. DatasetCreator should be used when users prefer to specify the per-replica batching and sharding logic for the Dataset. See tf.keras.utils.experimental.DatasetCreator doc for more information. A more detailed description of unpacking behavior for iterator types (Dataset, generator, Sequence) is given below. If using tf.distribute.experimental.ParameterServerStrategy, only DatasetCreator type is supported for x.
* y: Target data. Like the input data x, it could be either Numpy array(s) or TensorFlow tensor(s). It should be consistent with x (you cannot have Numpy inputs and tensor targets, or inversely). If x is a dataset, generator, or keras.utils.Sequence instance, y should not be specified (since targets will be obtained from x).
* batch_size: Integer or None. Number of samples per gradient update. If unspecified, batch_size will default to 32. Do not specify the batch_size if your data is in the form of datasets, generators, or keras.utils.Sequence instances (since they generate batches).
* epochs: Integer. Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided. Note that in conjunction with initial_epoch, epochs is to be understood as "final epoch". The model is not trained for a number of iterations given by epochs, but merely until the epoch of index epochs is reached.
* verbose: 'auto', 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy. Note that the progress bar is not particularly useful when logged to a file, so verbose=2 is recommended when not running interactively (eg, in a production environment).
* callbacks: List of keras.callbacks.Callback instances. List of callbacks to apply during training. See tf.keras.callbacks. Note tf.keras.callbacks.ProgbarLogger and tf.keras.callbacks.History callbacks are created automatically and need not be passed into model.fit. tf.keras.callbacks.ProgbarLogger is created or not based on verbose argument to model.fit. Callbacks with batch-level calls are currently unsupported with tf.distribute.experimental.ParameterServerStrategy, and users are advised to implement epoch-level calls instead with an appropriate steps_per_epoch value.
* validation_split: Float between 0 and 1. Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. The validation data is selected from the last samples in the x and y data provided, before shuffling. This argument is not supported when x is a dataset, generator or keras.utils.Sequence instance. validation_split is not yet supported with tf.distribute.experimental.ParameterServerStrategy.
* validation_data: Data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data. Thus, note the fact that the validation loss of data provided using validation_split or validation_data is not affected by regularization layers like noise and dropout. validation_data will override validation_split. validation_data could be: - A tuple (x_val, y_val) of Numpy arrays or tensors. - A tuple (x_val, y_val, val_sample_weights) of NumPy arrays. - A tf.data.Dataset. - A Python generator or keras.utils.Sequence returning (inputs, targets) or (inputs, targets, sample_weights). validation_data is not yet supported with tf.distribute.experimental.ParameterServerStrategy.
* shuffle: Boolean (whether to shuffle the training data before each epoch) or str (for 'batch'). This argument is ignored when x is a generator or an object of tf.data.Dataset. 'batch' is a special option for dealing with the limitations of HDF5 data; it shuffles in batch-sized chunks. Has no effect when steps_per_epoch is not None.
* class_weight: Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
* sample_weight: Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only). You can either pass a flat (1D) Numpy array with the same length as the input samples (1:1 mapping between weights and samples), or in the case of temporal data, you can pass a 2D array with shape (samples, sequence_length), to apply a different weight to every timestep of every sample. This argument is not supported when x is a dataset, generator, or keras.utils.Sequence instance, instead provide the sample_weights as the third element of x.
* initial_epoch: Integer. Epoch at which to start training (useful for resuming a previous training run).
* steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. If x is a tf.data dataset, and 'steps_per_epoch' is None, the epoch will run until the input dataset is exhausted. When passing an infinitely repeating dataset, you must specify the steps_per_epoch argument. This argument is not supported with array inputs. steps_per_epoch=None is not supported when using tf.distribute.experimental.ParameterServerStrategy.
* validation_steps: Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch. If 'validation_steps' is None, validation will run until the validation_data dataset is exhausted. In the case of an infinitely repeated dataset, it will run into an infinite loop. If 'validation_steps' is specified and only part of the dataset will be consumed, the evaluation will start from the beginning of the dataset at each epoch. This ensures that the same validation samples are used every time.
* validation_batch_size: Integer or None. Number of samples per validation batch. If unspecified, will default to batch_size. Do not specify the validation_batch_size if your data is in the form of datasets, generators, or keras.utils.Sequence instances (since they generate batches).
* validation_freq: Only relevant if validation data is provided. Integer or collections.abc.Container instance (e.g. list, tuple, etc.). If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs. If a Container, specifies the epochs on which to run validation, e.g. validation_freq=[1, 2, 10] runs validation at the end of the 1st, 2nd, and 10th epochs.
* max_queue_size: Integer. Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
* workers: Integer. Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
* use_multiprocessing: Boolean. Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. Note that because this implementation relies on multiprocessing, you should not pass non-picklable arguments to the generator as they can't be passed easily to children processes.

In [204]:
# x: input
# y: target
# epochs number of epochs to train the model
# verbose: for 0: don't show result ,for 1: step to step show result , for 2:every complete epoch then show result
# val_split: Model, eğitim verilerinin bu bölümünü ayıracak, 
# bunun üzerinde çalışmayacak ve her dönemin sonunda bu verilerdeki kaybı ve model ölçütlerini değerlendirecektir. 
history = model.fit([train.userId, train.movieId], train.rating,  epochs=10,batch_size=64,verbose=1, validation_split=0.1)

Epoch 1/10
11797/11797 [==============================] - 7s 527us/step - loss: 0.7632 - accuracy: 0.0332 - val_loss: 0.6822 - val_accuracy: 0.0332
Epoch 2/10
11797/11797 [==============================] - 6s 519us/step - loss: 0.6794 - accuracy: 0.0336 - val_loss: 0.6733 - val_accuracy: 0.0331
Epoch 3/10
11797/11797 [==============================] - 6s 518us/step - loss: 0.6740 - accuracy: 0.0337 - val_loss: 0.6755 - val_accuracy: 0.0332
Epoch 4/10
11797/11797 [==============================] - 6s 517us/step - loss: 0.6733 - accuracy: 0.0336 - val_loss: 0.6801 - val_accuracy: 0.0331
Epoch 5/10
11797/11797 [==============================] - 6s 520us/step - loss: 0.6721 - accuracy: 0.0332 - val_loss: 0.6832 - val_accuracy: 0.0329
Epoch 6/10
11797/11797 [==============================] - 6s 520us/step - loss: 0.6690 - accuracy: 0.0330 - val_loss: 0.6840 - val_accuracy: 0.0328
Epoch 7/10
11797/11797 [==============================] - 6s 520us/step - loss: 0.6707 - accuracy: 0.0335 - val_

In [205]:
a=model.evaluate([train.userId, train.movieId], train.rating , batch_size=128)

6554/6554 [==============================] - 2s 374us/step - loss: 0.6507 - accuracy: 0.0335


In [206]:
a

[0.6507437825202942, 0.03351691737771034]

In [207]:
y_hat = np.round(model.predict([test.userId, test.movieId]),0)
y_true = test.rating

In [208]:
mean_absolute_error(y_true, y_hat)

0.6598526571775982

In [209]:
errors=mean_absolute_error(y_true, y_hat)
mape = 100 * (errors / y_true)
accuracy = 100 - np.mean(mape)

In [211]:
print("(MAE)Mean Absolute Error:",round(mean_absolute_error(y_true, y_hat),4))
print("(RMSE)Root Mean Square Error:",round(np.sqrt(mean_squared_error(y_true,y_hat)) ,4))
print(' Accuracy:', round(accuracy, 2), '%.')

(MAE)Mean Absolute Error: 0.6599
(RMSE)Root Mean Square Error: 0.9252
 Accuracy: 77.63 %.


In [214]:
mlp_user_embedding_weights = (next(iter(filter(lambda x: x.name == 'User-Embedding', model.layers))).get_weights())

In [219]:
print(mlp_user_embedding_weights)
len(mlp_user_embedding_weights[0])

[array([[  0.52441335],
       [ -8.120131  ],
       [ -8.188935  ],
       ...,
       [ -1.8790382 ],
       [-11.651901  ],
       [  0.01544632]], dtype=float32)]


7121

In [220]:
user_latent_matrix = mlp_user_embedding_weights[0]

In [221]:
user_latent_matrix

array([[  0.52441335],
       [ -8.120131  ],
       [ -8.188935  ],
       ...,
       [ -1.8790382 ],
       [-11.651901  ],
       [  0.01544632]], dtype=float32)

In [222]:
desired_user_id=38

In [223]:
one_user_vector = user_latent_matrix[desired_user_id,:]

In [224]:
one_user_vector

array([-5.092189], dtype=float32)

In [225]:
one_user_vector = np.reshape(one_user_vector, (1,1))

In [226]:
one_user_vector

array([[-5.092189]], dtype=float32)

In [227]:
print('Performing kmeans to find the nearest users...')

Performing kmeans to find the nearest users...


In [228]:
kmeans = KMeans(n_clusters=20, random_state=0, verbose=0).fit(user_latent_matrix)

In [229]:
kmeans

KMeans(n_clusters=20, random_state=0)

In [230]:
desired_user_label = kmeans.predict(one_user_vector)

In [231]:
desired_user_label

array([13])

In [232]:
user_label = kmeans.labels_

In [233]:
user_label

array([5, 6, 6, ..., 8, 3, 5])

In [235]:
neighbors = []
for user_id, user_label in enumerate(user_label):
    if user_label == desired_user_label:
        neighbors.append(user_id)

In [236]:
neighbors

[7,
 30,
 33,
 38,
 43,
 56,
 57,
 61,
 62,
 66,
 67,
 88,
 98,
 117,
 139,
 158,
 200,
 210,
 235,
 243,
 248,
 252,
 255,
 267,
 274,
 284,
 286,
 295,
 308,
 324,
 351,
 360,
 404,
 442,
 443,
 445,
 473,
 477,
 483,
 495,
 515,
 533,
 534,
 537,
 548,
 588,
 594,
 627,
 638,
 640,
 641,
 650,
 655,
 664,
 667,
 669,
 672,
 677,
 682,
 705,
 726,
 782,
 796,
 801,
 828,
 840,
 844,
 849,
 862,
 867,
 869,
 885,
 899,
 926,
 932,
 938,
 946,
 965,
 1024,
 1028,
 1036,
 1041,
 1042,
 1073,
 1088,
 1105,
 1116,
 1147,
 1155,
 1157,
 1165,
 1181,
 1215,
 1229,
 1234,
 1241,
 1245,
 1250,
 1254,
 1258,
 1262,
 1282,
 1292,
 1294,
 1296,
 1330,
 1341,
 1380,
 1381,
 1404,
 1426,
 1427,
 1467,
 1481,
 1486,
 1499,
 1500,
 1502,
 1503,
 1504,
 1505,
 1511,
 1533,
 1538,
 1540,
 1544,
 1552,
 1557,
 1574,
 1583,
 1605,
 1619,
 1626,
 1629,
 1630,
 1632,
 1635,
 1648,
 1654,
 1664,
 1667,
 1668,
 1675,
 1695,
 1715,
 1723,
 1746,
 1758,
 1789,
 1792,
 1799,
 1811,
 1824,
 1834,
 1835,
 1838,


In [237]:
print('Found {0} neighbor users.'.format(len(neighbors)))

Found 591 neighbor users.


In [238]:
movies = []
for user_id in neighbors:
    movies += list(ratings[ratings['userId'] == int(user_id)]['movieId'])
    

In [239]:
movies = list(set(movies))

In [242]:
movies

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 51,
 53,
 54,
 56,
 57,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 104,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 117,
 118,
 119,
 120,
 121,
 122,
 128,
 131,
 136,
 137,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,

In [243]:
print('Found {0} neighbor movies from these users.'.format(len(movies)))

Found 6988 neighbor movies from these users.


In [244]:
users = np.full(len(movies), desired_user_id, dtype='int32')


In [245]:
users

array([38, 38, 38, ..., 38, 38, 38])

In [247]:
items = np.array(movies, dtype='int32')

In [248]:
items

array([    0,     1,     2, ..., 13975, 13992, 13993])

In [249]:
print('Ranking most likely tracks using the NeuMF model...')

Ranking most likely tracks using the NeuMF model...


In [263]:
results = model.predict([users,items], verbose=1)

219/219 [==============================] - 0s 590us/step


In [264]:
results = results.tolist()

In [265]:
results

[[4.2146148681640625],
 [3.6068098545074463],
 [3.508782148361206],
 [3.3198158740997314],
 [3.4155936241149902],
 [4.202577114105225],
 [3.6695895195007324],
 [3.6419565677642822],
 [3.5633862018585205],
 [3.7226738929748535],
 [3.925507068634033],
 [3.097158432006836],
 [3.555128335952759],
 [3.6436924934387207],
 [3.2605199813842773],
 [4.153736114501953],
 [4.227509498596191],
 [3.8633205890655518],
 [2.952104091644287],
 [3.340705394744873],
 [3.969735622406006],
 [3.6698596477508545],
 [3.453993082046509],
 [3.6129696369171143],
 [4.063234329223633],
 [4.110652923583984],
 [3.717965841293335],
 [4.335838317871094],
 [4.390641689300537],
 [4.019095420837402],
 [3.563668966293335],
 [4.180771827697754],
 [4.03408670425415],
 [4.281841278076172],
 [4.101990222930908],
 [3.243173837661743],
 [3.824474334716797],
 [3.9840428829193115],
 [4.151639461517334],
 [3.4942774772644043],
 [3.820448875427246],
 [3.313750743865967],
 [3.7941360473632812],
 [3.5203919410705566],
 [4.454586982727

In [266]:
print('Ranked the movies!')

Ranked the movies!


In [267]:
results = pd.DataFrame(results, columns=['pre_rating']).astype("float")

In [268]:
results

,pre_rating
0,4.214615
1,3.606810
2,3.508782
3,3.319816
4,3.415594
...,...
6983,3.931667
6984,3.119883
6985,1.334036
6986,3.931763


In [269]:
items = pd.DataFrame(items, columns=['movieId'])

In [270]:
items

,movieId
0,0
1,1
2,2
3,3
4,4
...,...
6983,13953
6984,13955
6985,13975
6986,13992


In [271]:
results = pd.concat([items, results], ignore_index=True, sort=False, axis=1)

In [273]:
results

,0,1
0,0,4.214615
1,1,3.606810
2,2,3.508782
3,3,3.319816
4,4,3.415594
...,...,...
6983,13953,3.931667
6984,13955,3.119883
6985,13975,1.334036
6986,13992,3.931763


In [274]:
results.columns =['movieId', 'pre_rating']

In [275]:
results

,movieId,pre_rating
0,0,4.214615
1,1,3.606810
2,2,3.508782
3,3,3.319816
4,4,3.415594
...,...,...
6983,13953,3.931667
6984,13955,3.119883
6985,13975,1.334036
6986,13992,3.931763


In [276]:
results_df = pd.DataFrame(np.nan, index=range(len(results)), columns=['pre_rating','movieId'])

In [277]:
results_df

,pre_rating,movieId
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
6983,NaN,NaN
6984,NaN,NaN
6985,NaN,NaN
6986,NaN,NaN


In [278]:
for index, row in results.iterrows():
        results_df.loc[index] = [row['pre_rating'], ratings[ratings['movieId'] == row['movieId']].iloc[0]['movieId']]

In [279]:
results_df

,pre_rating,movieId
0,4.214615,0.0
1,3.606810,1.0
2,3.508782,2.0
3,3.319816,3.0
4,3.415594,4.0
...,...,...
6983,3.931667,13953.0
6984,3.119883,13955.0
6985,1.334036,13975.0
6986,3.931763,13992.0


In [280]:
results_df= results_df.sort_values(by=['pre_rating'], ascending=False)

In [281]:
results_df

,pre_rating,movieId
6161,5.138682,10476.0
4827,5.042960,7093.0
5935,5.037367,9965.0
6616,4.982715,12013.0
6800,4.975094,12742.0
...,...,...
3143,1.502903,4012.0
6985,1.334036,13975.0
1347,1.285198,1694.0
2841,1.163361,3606.0


In [282]:
results_df["movieId"]=results_df["movieId"].astype(int)

In [283]:
results_df

,pre_rating,movieId
6161,5.138682,10476
4827,5.042960,7093
5935,5.037367,9965
6616,4.982715,12013
6800,4.975094,12742
...,...,...
3143,1.502903,4012
6985,1.334036,13975
1347,1.285198,1694
2841,1.163361,3606


In [284]:
results_df=pd.merge(results_df,movie_list,on="movieId")[:10]

In [285]:
results_df

,pre_rating,movieId,title,genres,rtPictureURL
0,5.042960,7093,"Front Page, The (1974)",Comedy,http://content9.flixster.com/movie/10/91/11/10...
1,4.966232,7170,My Baby's Daddy (2004),Comedy,http://content8.flixster.com/movie/26/67/26671...
2,4.960918,7813,FM (1978),Drama,http://content6.flixster.com/movie/26/43/26430...
3,4.889987,5608,"Das Experiment (Experiment, The) (2001)",Drama|Thriller,http://content7.flixster.com/movie/27/61/27619...
4,4.871102,8829,Benji: Off the Leash! (2004),Children|Drama,http://content9.flixster.com/movie/26/32/26322...
5,4.868045,3053,"Messenger: The Story of Joan of Arc, The (1999)",Drama|War,http://content6.flixster.com/movie/10/88/57/10...
6,4.856533,7838,"Internecine Project, The (1974)",Action|Thriller,http://content9.flixster.com/movie/10/95/79/10...
7,4.853911,8157,Jin Roh: The Wolf Brigade (Jin-Rô) (1998),Animation|Fantasy|Thriller,http://content6.flixster.com/movie/27/71/27710...
8,4.843448,7980,"Bridge Too Far, A (1977)",Action|Drama|War,http://content6.flixster.com/movie/25/70/25701...
9,4.842775,4957,Sudden Impact (1983),Crime|Thriller,http://content9.flixster.com/movie/10/88/98/10...
